In [45]:
import pyaes

#################################STAGE 1###############################################

# A 128 bit (16 byte) key
key = bytes.fromhex('00' * 16)
print(len(key))
# For some modes of operation we need a random initialization vector
# of 16 bytes
iv = "CAFEBABEFACEDBADDECAF888"
iv = '00' * 12
iv = bytes.fromhex(iv)

aes = pyaes.AESModeOfOperationECB(key)
plaintext = bytes.fromhex("00"* 16)
H = ciphertext = aes.encrypt(plaintext)
# H.hex()

16


In [79]:
import pyaes
import math

class AES_GCM_128:
    
    def __init__(self, i_key):
        assert len(i_key) == 16, 'This class supports 128-bit key only!'
        self.i_key = i_key
        
        # Plain text and aad size
        self.plt_size = 0
        self.aad_size = 0
            
    def update_i_key(self, new_key):
        self.i_key = new_key
        
    @property
    def aes(self):
        return pyaes.AESModeOfOperationECB(self.i_key)
        
    @property
    def H(self):
        return self.aes().encrypt(bytes.fromhex("00"* 16))
         
    @staticmethod
    def xor(a, b):
        return bytes([aa^bb for aa, bb in zip(a, b)])

    def gctr(self, iv, x, phase='encrypt'):
        cb = []
        offset = 2 if phase == 'encrypt' else 1
#         print(len(iv))
        for i in range(10):
#             print(iv + b'\x00' * 3 + bytes([i]))
            cb.append(iv + b'\x00' * 3 + bytes([i]))
        if x == "":
            return ''
        y = []
        n = math.ceil(len(x) / 16)
        for j in range(n):
            cb_ = cb[j + offset]
#             print('cb_ length ', len(cb_))
            ciphed = self.aes.encrypt(cb_)
            y.append(xor(x[j* 16 : j* 16 + 16], ciphed))
        return y
    
    @staticmethod
    def shiftright(a):
        leng = len(a)
        carry = 0
        res = []
        for i in range(leng):
            swap = a[i] % 2
            res.append(a[i] // 2 + carry * (pow(2, 7)))
            carry = swap
        return bytes(res)
    
    @staticmethod
    def hashmul(x, y):
        z = b'\x00' * 16 
        v = y
        ele = b'\xe1' + b'\x00' * 15

        for i in x:
            num = i
            for j in range(8):
                div = pow(2, 7 - j)
                here = num // div
                num = num % div

                if here == 1:
                    z = xor(z, v)
                if v[-1] % 2 == 0:
                    v = shiftright(v)
                else:
                    v = shiftright(v)
                    v = xor(v, ele)
        return z

    def cal_size(self, aad, pt):
        self.aad_size = (len(aad) * 8).to_bytes(8, 'big')
        self.plt_size = (len(pt) * 8).to_bytes(8, 'big')
        
    def encrypt(self, iv, aad, pt):
        # Encrypt the text
        self.cal_size(aad, pt)
        j0 = iv + b'00' * 3 + b'01'
        yy = self.gctr(iv, pt)
        C= b''
        for y in yy:
            C = C+ y  
        
        # Generate the tag
        y1 = b'\x00' * 16
        xx = aad + C + self.aad_size + self.plt_size
        for i in range(len(xx) // 16):
            y1 = hashmul(xor(xx[i * 16:i * 16 +16], y1), H)
        S = y1
        t = self.gctr(iv, S, 'tag')
        T = t[0]
        return C, T
        
if __name__ == '__main__':
    key = bytes.fromhex('00' * 16)
    iv = bytes.fromhex('00' * 12)
    pt = bytes.fromhex('00' * 32)
    aad = bytes.fromhex('00' * 16)

    gcm_aes = AES_GCM_128(key)
    C, t = gcm_aes.encrypt(iv, aad, pt)
    print( C.hex(), T.hex())

0388dace60b6a392f328c2b971b2fe78f795aaab494b5923f7fd89ff948bc1e0 39691d1ab70a22084ae9337c697c3bee


In [59]:
#################################STAGE 2 - 3###############################################

# X is the plaintext
# ICB is the increased J0
# Key is key
pt = bytes.fromhex('D9313225 F88406E5 A55909C5 AFF5269A 86A7A953 1534F7DA 2E4C303D 8A318A72 1C3C0C95 95680953 2FCF0E24 49A6B525 B16AEDF5 AA0DE657 BA637B39 1AAFD255')
aad = bytes.fromhex('3AD77BB4 0D7A3660 A89ECAF3 2466EF97 F5D3D585 03B9699D E785895A 96FDBAAF 43B1CD7F 598ECE23 881B00E3 ED030688 7B0C785E 27E8AD3F 82232071 04725DD4')
# key = '00' * 16

pt = bytes.fromhex('00' * 32)
aad = bytes.fromhex('00' * 16)

j0 = iv + b'00' * 3 + b'01'
import math
def xor(a, b):
    return bytes([aa^bb for aa, bb in zip(a, b)])
        
def gctr(icb, key, x):
    cb = []
    for i in range(16):
        cb.append(iv + b'\x00' * 3 + bytes([i]))
    if x == "":
        return ''
    y = []
    n = math.ceil(len(x)/16)
#     print(cb[2])
    for j in range(n):
        cb_ = cb[j + 2]
        ciphed = aes.encrypt(cb_)
        y.append(xor(x[j* 16 : j* 16 + 16], ciphed))
    return y
    
yy = gctr(j0, key, pt)
C= b''
for y in yy:
    C = C+ y
C.hex()

'0388dace60b6a392f328c2b971b2fe78f795aaab494b5923f7fd89ff948bc1e0'

In [47]:
#################################STAGE 4 5 6###############################################
# u = aad + C + 3 * b'\x00' + bytes()
def shiftright(a):
    leng = len(a)
    carry = 0
    res = []
    for i in range(leng):
        swap = a[i] % 2
        res.append(a[i] // 2 + carry * (pow(2, 7)))
        carry = swap
    return bytes(res)
        
def gctr1(icb, key, x):
    cb = []
    for i in range(16):
        cb.append(iv + b'\x00' * 3 + bytes([i]))
    if x == "":
        return ''
    y = []
    n = math.ceil(len(x)/16)
#     print(cb[2])
    for j in range(n):
        cb_ = cb[j + 1]
        ciphed = aes.encrypt(cb_)
        y.append(xor(x[j* 16 : j* 16 + 16], ciphed))
    return y

def hashmul(x, y):
    z = b'\x00' * 16 
    v = y
    ele = b'\xe1' + b'\x00' * 15
    
    for i in x:
        num = i
        for j in range(8):
            div = pow(2, 7 - j)
            here = num // div
            num = num % div
            
            if here == 1:
                z = xor(z, v)
            if v[-1] % 2 == 0:
                v = shiftright(v)
            else:
                v = shiftright(v)
                v = xor(v, ele)
    return z
y1 = b'\x00' * 16
xx = aad + C + b'\x00' * 7 + b'\x80' +  b'\x00' * 6 + b'\x01'  + b'\x00' 

for i in range(4):
    y1 = hashmul(xor(xx[i * 16:i * 16 +16], y1), H)
S = y1
t = gctr1(j0, key, S)
T = t[0]

In [82]:
print('S is\n\t\t', S.hex())
print('\nC is\t\t')
for i in range((len(pt) * 8).to_bytes(8, 'big') / 128):
    print('\t\t', C[i * 16:i * 16 +16].hex())
print('\nTag is\n','\t\t',T.hex())

S is
		 618be1d44d7412697c962e2bcd9b7eb4

C is		


TypeError: unsupported operand type(s) for /: 'bytes' and 'int'

In [53]:
def cal_size(aad, pt):
    print(len(aad))
    print(len(pt))
    return (len(pt) * 8).to_bytes(8, 'big')
pt = bytes.fromhex('00' * 32)
aad = bytes.fromhex('00' * 16)   
cal_size(aad, pt)

16
32


b'\x00\x00\x00\x00\x00\x00\x01\x00'